In [49]:
import pandas as pd
import numpy as np
from config import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import ParameterEstimator, MaximumLikelihoodEstimator



/home/cjq/anaconda3/envs/pgm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
# read csv
df = pd.read_csv("./dataset/preprocessed_data.csv", index_col=None)
df = df.drop(["CODE_GENDER", "STATUS", "ID"], axis=1)

In [38]:
df.isnull().sum()

FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
AMT_INCOME_TOTAL       0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
DAYS_BIRTH             0
DAYS_EMPLOYED          0
FLAG_MOBIL             0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
MONTHS_BALANCE         0
GOOD_REPUTATION        0
dtype: int64

In [39]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(df["OCCUPATION_TYPE"])
df["OCCUPATION_TYPE"] = encoder.transform(df["OCCUPATION_TYPE"])
for colum in df.columns:
    print(pd.unique(df[colum]))

[1 0]
[1 0]
[ 0  1  3  2  4  5 14 19  7]
[9 1 8 2 4 6 7 5 0 3]
[5 3 1]
[4 2 3 1 5]
[3 1 2]
[1 5 4 0 3 2]
[7 1 2 0 4 3 8 9 5 6]
[1 5 2 8 6 4 0 3 7]
[1]
[1 0]
[0 1]
[0 1]
[17 16 14  0  8 10  4  3  6  1 12  2  9 11 15 18  5 13  7]
[ 2.  1.  5.  3.  4.  6. 15.  7. 20.  9.]
[  0 -22  -4  -1 -10  -8 -14  -3 -28  -2  -9 -11 -40 -24 -38  -7 -18 -15
 -32  -6 -43 -20 -19 -17 -12 -33  -5 -51 -47 -21 -34 -29 -35 -49 -31 -42
 -23 -44 -37 -13 -46 -25 -39 -56 -36 -16 -26 -27 -48 -41 -30 -45 -55 -52
 -57 -50 -53 -54 -59 -60 -58]
[0 1]


In [40]:
df_features = df.iloc[:, :-1]
df_label = df.iloc[:, -1]

In [41]:
from sklearn.feature_selection import mutual_info_classif
info = mutual_info_classif(df_features, df_label)

In [44]:
column_info = {"columns": df_features.columns.tolist(), "info": info}

In [48]:
pd.DataFrame.from_dict(column_info).sort_values("info", ascending=False).reset_index(drop=True)

,columns,info
0,NAME_HOUSING_TYPE,0.010109
1,FLAG_OWN_REALTY,0.007344
2,NAME_FAMILY_STATUS,0.006217
3,NAME_EDUCATION_TYPE,0.005436
4,FLAG_MOBIL,0.005086
5,OCCUPATION_TYPE,0.004876
6,NAME_INCOME_TYPE,0.004577
7,FLAG_WORK_PHONE,0.003745
8,FLAG_OWN_CAR,0.002383
9,DAYS_BIRTH,0.002295


In [ ]:
from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

In [ ]:
print_full(model.get_cpds('Risk'))

+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Checking account | Checking account(little) | Checking account(moderate) | Checking account(rich) | Checking account(unknown) |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Risk(0)          | 0.4959016393442623       | 0.6056910569105691         | 0.7758620689655172     | 0.8721590909090909        |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+
| Risk(1)          | 0.5040983606557377       | 0.3943089430894309         | 0.22413793103448276    | 0.1278409090909091        |
+------------------+--------------------------+----------------------------+------------------------+---------------------------+


In [ ]:
from itertools import combinations, permutations

for combo in permutations(df_features.columns.tolist(), 2):
    # Create a Bayesian model with the current combination
    print(combo)
    model = BayesianNetwork([combo])
    
    model.fit(data, estimator=MaximumLikelihoodEstimator)

    # Print the combination and the learned parameters (CPDs)
    print(f"Combination: {combo}")
    for cpd in model.get_cpds():
        print(cpd)
    print("\n" + "-" * 50 + "\n") 
